In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decpomdp import DecPOMDP
import Classes
from constant import Constants
import time
import os



In [11]:
file_name = "Mars"
planning_horizon = 3
num_iterations = 1


In [12]:
#import problem
problem = DecPOMDP(file_name, 1,horizon=planning_horizon)
# constant = Constants(problem)
# utilities = Utilities(constant)
# Classes.set_problem(problem)

# # print(f"game of initiated with SOTA set to = {sota_} with horizon {planning_horizon}")
print("GAME DESCRIPTION :")
print(f"game size :\n\t|S| = {len(problem.states)}")
print(f"\t|Z| = {problem.num_joint_observations}\n\t|U| = {problem.num_joint_actions} with |U_i| = {problem.num_actions[0]}")
print(f"intiial_belief : {problem.b0}")


print(f"{problem.num_actions[0]},{problem.num_actions[0]}")

GAME DESCRIPTION :
game size :
	|S| = 256
	|Z| = 64
	|U| = 36 with |U_i| = 6
intiial_belief : [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
6,6


In [4]:
# solve
def SOLVE(game):
    start_time = time.time()
    policy = game.solve(num_iterations,0.9)
    end_time = time.time()
    solve_time = end_time - start_time
    value_fn = game.value_function
    return policy,solve_time,value_fn

In [5]:
def initialize_database():
    database = {"gametype":[],
                "SOTA" : [],
                "horizon": [],
                    "num_iterations" : [],
                    "average_time" : [],
                    "number_of_beliefs" : [],
                    "ave_leader_value_b0":[],
                    "ave_follower_value_b0":[]
                    # "density" = []
                    # "gap":[]
                   
                    }
    return database

def add_to_database(database,horizon,game_type,num_iterations,average_time,num_beliefs,V0_B0,V1_B0,SOTA):
    sota = {True:"State of the Art" , False:"Stackelberg"}
    database["gametype"].append(game_type)
    database["horizon"].append(horizon)
    database["SOTA"].append(sota[SOTA])
    database["num_iterations"].append(num_iterations)
    database["average_time"].append(average_time)
    database["number_of_beliefs"].append(num_beliefs)
    database["ave_leader_value_b0"].append(V0_B0)
    database["ave_follower_value_b0"].append(V1_B0)
    # database["gap"].append(abs(V0_B0-V1_B0))
    # database["density"].append(density)
    return


In [6]:
database = initialize_database()
for gametype in ["cooperative","zerosum","stackelberg"]:
    for horizon in range(1,planning_horizon+1):
        problem = DecPOMDP(file_name,1,horizon=planning_horizon)
        Classes.set_problem(problem)
        for sota_ in [False,True]:
            print(f"\n============= {gametype} GAME WITH HORIZON {horizon} , SOTA {sota_} ===========")
          
            game = Classes.PBVI(problem=problem,horizon=horizon,density=0.1,gametype=gametype,sota=sota_)
            policy, time_ , value_fn = SOLVE(game)
            num_beliefs = game.belief_space.belief_size()
            value0,value1= value_fn.get_values_initial_belief()
            add_to_database(database,horizon,gametype,2,time_,num_beliefs,value0,value1,sota_)
#POLICY PRINTING: 
# print("\nLEADER POLICY\n")
# policy[0].print_trees()
# print("\nFOLLOWER POLICY\n")
# policy[1].print_trees()        



database = pd.DataFrame(database)
path = "Results/"
file_name = f"{file_name}_{horizon}_{num_iterations}"
database.to_csv(file_name, index=False)



problem set to dectiger

============= cooperative GAME WITH HORIZON 1 , SOTA False ===========
	belief expansion done
iteration : 1


	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 1 , SOTA True ===========
	belief expansion done
iteration : 1
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
problem set to dectiger

============= cooperative GAME WITH HORIZON 2 , SOTA False ===========
	belief expansion done
iteration : 1
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= cooperative GAME WITH HORIZON 2 , SOTA True ===========
	belief expansion done
iteration : 1
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done
problem set to dectiger

============= cooperative GAME WITH HORIZON 3 , SOTA False ===========
	belief expansion done
iteration : 1
	backup at timestep 3 done
	backup at timestep 2 done
	backup at timestep 1 done
	backup at timestep 0 done
	backward induction done

============= 

In [7]:
database

,gametype,SOTA,horizon,num_iterations,average_time,number_of_beliefs,ave_leader_value_b0,ave_follower_value_b0
0,cooperative,Stackelberg,1,2,0.080060,4,[7.5],[7.5]
1,cooperative,State of the Art,1,2,0.006375,4,[7.5],[7.5]
2,cooperative,Stackelberg,2,2,0.164079,9,[8.815],[8.815]
3,cooperative,State of the Art,2,2,0.020593,9,[8.815],[8.815]
4,cooperative,Stackelberg,3,2,0.310790,14,[15.000000000000004],[15.000000000000004]
5,cooperative,State of the Art,3,2,0.022244,14,[15.000000000000004],[15.000000000000004]
6,zerosum,Stackelberg,1,2,0.195428,4,[-74.63999999999999],[74.63999999999999]
7,zerosum,State of the Art,1,2,0.111575,4,[-74.63999999999999],[-74.63999999999999]
8,zerosum,Stackelberg,2,2,0.176122,9,[-103.27999999999999],[103.27999999999999]
9,zerosum,State of the Art,2,2,0.209251,9,[-103.27999999999999],[-103.27999999999999]


game.value_function

In [8]:
for i in game.value_function.vector_sets[0]:
    print(i.get_value(game.belief_space.get_inital_belief()))

(-2.113130722099695, -52.611111111111114)


In [9]:
print(f"RESULTS FOR {file_name}:\n")

RESULTS FOR dectiger_3_<built-in function iter>:

